# **Penting**
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Hapus simbol pagar (#) jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [85]:
#Type your code here
from joblib import dump
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [86]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
df = pd.read_csv('./data_clustering_inverse.csv')

In [87]:
# Tampilkan 5 baris pertama dengan function head.
df.head(5)

,TransactionAmount,TransactionType,Channel,CustomerAge,CustomerOccupation,TransactionDuration,AccountBalance,CustomerAge_encoded,Target
0,84,0,0,52,0,71,916,4,1
1,1309,0,0,50,0,131,1707,4,1
2,636,0,2,1,3,46,327,0,0
3,836,0,2,8,3,15,1422,1,0
4,484,0,0,0,3,162,195,0,0


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [88]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop(columns=['Target','CustomerAge_encoded'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f'total data: {X.shape[0]}')
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')


total data: 1760
X_train shape: (1408, 7)
X_test shape: (352, 7)


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [89]:
# Buatlah model klasifikasi menggunakan Decision Tree
model_dt = DecisionTreeClassifier().fit(X_train, y_train)


In [90]:
# Menyimpan Model
# import joblib
# joblib.dump(model, 'decision_tree_model.h5')
dump(model_dt, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [91]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.
model_rf = RandomForestClassifier().fit(X_train, y_train)

In [92]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
def evaluate_model(model, X_test, y_test):
  y_pred = model.predict(X_test)
  results = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='micro'),
    'Recall': recall_score(y_test, y_pred, average='micro'),
    'F1-Score': f1_score(y_test, y_pred, average='micro')
  }
  return results

results = {
  'Decision Tree (DT)': evaluate_model(model_dt, X_test, y_test),
  'Random Forest (RF)': evaluate_model(model_rf, X_test, y_test)
}

summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

rows = []
for model_name, metrics in results.items():
  rows.append({
    'Model': model_name,
    'Accuracy': metrics['Accuracy'],
    'Precision': metrics['Precision'],
    'Recall': metrics['Recall'],
    'F1-Score': metrics['F1-Score']
  })
  
summary_df = pd.DataFrame(rows)

summary_df

,Model,Accuracy,Precision,Recall,F1-Score
0,Decision Tree (DT),0.982955,0.982955,0.982955,0.982955
1,Random Forest (RF),0.974432,0.974432,0.974432,0.974432


In [93]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
# import joblib
# joblib.dump(___, 'explore_<Nama Algoritma>_classification.h5')
dump(model_rf, 'explore_random_forest_classification.h5')

['explore_random_forest_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [94]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
params = {
    'n_estimators': [100, 500],
    'max_depth': [2, 20],
    'min_samples_split': [2, 20],
    'min_samples_leaf': [1, 20],
    'max_features': [2, 11]
}
modelRfForTuning = RandomForestClassifier(random_state=42)

model_gs = GridSearchCV(estimator=modelRfForTuning, param_grid=params, cv=3, n_jobs=-1, verbose=2)
model_gs.fit(X_train, y_train)

print(f"Best parameters (Grid Search): {model_gs.best_params_}")
best_modelRf = model_gs.best_estimator_

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best parameters (Grid Search): {'max_depth': 20, 'max_features': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [95]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
results = {
  'Grid Search (GS)': evaluate_model(model_gs, X_test, y_test),
  'Random Forest (RF)': evaluate_model(model_rf, X_test, y_test)
}

summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

rows = []
for model_name, metrics in results.items():
  rows.append({
    'Model': model_name,
    'Accuracy': metrics['Accuracy'],
    'Precision': metrics['Precision'],
    'Recall': metrics['Recall'],
    'F1-Score': metrics['F1-Score']
  })
  
summary_df = pd.DataFrame(rows)

summary_df

,Model,Accuracy,Precision,Recall,F1-Score
0,Grid Search (GS),0.985795,0.985795,0.985795,0.985795
1,Random Forest (RF),0.974432,0.974432,0.974432,0.974432


In [96]:
# Menyimpan Model hasil tuning
# import joblib
# joblib.dump(model_dt, 'tuning_classification.h5')
dump(best_modelRf, 'tuning_classification.h5')

['tuning_classification.h5']